In [2]:
#imports gerais
import sys
import os
from pathlib import Path
import re
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm import tqdm

In [3]:
# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
#plot
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS

     |████████████████████████████████| 1.7 MB 12.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=70e6c101de2f816945d9bda8d993a2f356b7aeab635b69ab91bf03b7def300c7
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [6]:
# montar google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df_noticias = pd.read_csv('/content/drive/My Drive/aneel-imprensa-2020/dados/dados_noticias_limpo.csv')

In [8]:
df_noticias.head()

Nome Veículo      Categoria Data Notícia  \
0  DIÁRIO DO GRANDE ABC - SP      REGULAÇÃO   2020-01-01   
1       O ESTADO DE S. PAULO  INSTITUCIONAL   2020-01-01   
2       AGÊNCIA O GLOBO - RJ  INSTITUCIONAL   2020-01-01   
3       AGÊNCIA O GLOBO - RJ        MERCADO   2020-01-01   
4       AGÊNCIA O GLOBO - RJ      REGULAÇÃO   2020-01-01   

                                        Título limpo  \
0         atila diminui orçamento arsep aumenta sama   
1  juro menor ajuda reduzir custos industriais ed...   
2  vem brazil water week evento internacional tra...   
3  polo multissetorial serviços vai mudar cara ba...   
4  polo multissetorial serviços vai mudar cara ba...   

                                 Texto Notícia limpo  
0  prefeito mauá atila jacomussi psb decidiu dar ...  
1  estrutural redução custos setor manufatureiro ...  
2  energia alternativa abes associação brasileira...  
3  economia bairro recife belas paisagens pernamb...  
4  economia bairro recife belas paisagens pernamb...

In [9]:
df_noticias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131878 entries, 0 to 131877
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Nome Veículo         131878 non-null  object
 1   Categoria            131878 non-null  object
 2   Data Notícia         131878 non-null  object
 3   Título limpo         131868 non-null  object
 4   Texto Notícia limpo  127536 non-null  object
dtypes: object(5)
memory usage: 5.0+ MB


In [10]:
#atribui lista pela coluna de análise desejada

list_noticias = df_noticias['Título limpo'].values.tolist()
#list_noticias = df_noticias['Texto Notícia limpo'].values.tolist()

In [11]:
# lista para retirar palavras comuns e óbvias
lista_palavras_comuns = ['energia',
                         'institucional',
                         'aneel',
                         'elétrica',
                         'nacional',
                         'agência',
                         'conta',
                         'consumidores',
                         'ser',
                         'estado',
                         'ainda',
                         'mês',
                         'sobre',
                         'desde',
                         'disse',
                         'segundo',
                         'pode',
                         'ano',
                         'mil',
                         'milhões',
                         'dia',
                         'nesta',
                         'hoje',
                         'paulo',
                         'mato',
                         'grosso',
                         'janeiro',
                         'fevereiro',
                         'março',
                         'abril',
                         'maio',
                         'junho',
                         'julho',
                         'agosto',
                         'setembro',
                         'outubro',
                         'novembro',
                         'dezembro',
                         'segunda',
                         'terça',
                         'quarta',
                         'quinta',
                         'sexta',
                         'sábado',
                         'domingo',
                         'feira',
                         'além',
                         'disso',
                         'empresas',
                         'desse',
                         'estadão',
                         'conteúdo',
                         'redação'
]

In [13]:
list_noticias

['atila diminui orçamento arsep aumenta sama',
 'juro menor ajuda reduzir custos industriais editorial econômico',
 'vem brazil water week evento internacional transformará paulo capital mundial água',
 'polo multissetorial serviços vai mudar cara bairro recife',
 'polo multissetorial serviços vai mudar cara bairro recife',
 'aquario vira referência energia limpa projeto faro energy',
 'aquario vira referência energia limpa projeto faro energy',
 'energia renovável leitor',
 'saiba quais projetos concessões leilões previstos',
 'saiba quais projetos concessões leilões previstos',
 'saiba quais projetos concessões leilões previstos',
 'caminhão derruba poste deixa milhares energia rondônia',
 'caminhão derruba poste deixa milhares energia rondônia',
 'valuengine upgrades workday nasdaq wday strong buy news',
 'tarifa branca clientes podem aderir economizar conta energia',
 'novidades sobre energia fotovoltaica nordeste',
 'consumidores baixa tensão podem aderir tarifa branca tentar redu

In [14]:
# tratamento com a palavra "novo" -> considerar as palavras "novos", "nova" e "novas" contabilizando como "novo"
for index, x in enumerate(list_noticias):
    x = str(x)
    list_noticias[index] = x.replace(' novos ', ' novo ')
    list_noticias[index] = x.replace(' nova ', ' novo ')
    list_noticias[index] = x.replace(' novas ', ' novo ')
    list_noticias[index] = x.replace(' pública ', ' públicos ')
    list_noticias[index] = x.replace(' primeira ', ' primeiro ')

In [16]:
#resolvendo alguns problemas de codificação
for index, x in enumerate(list_noticias):
    x = str(x)
    if '\x84' in x:
      list_noticias[index] = x.replace('\x84', 'ä')
    if '\x80' in x:
      list_noticias[index] = x.replace('\x80', 'ç')

In [18]:
# Apenas notícias que citam "aneel"
# Deixar flag abaixo como True para essa condição
fl_noticias_citam_aneel = False

if (fl_noticias_citam_aneel):
  list_noticias_aneel = []
  
  for index, x in enumerate(list_noticias):
    x = str(x)
    if (x.count('aneel') > 0):
      list_noticias_aneel.append(x)
  
  list_noticias = list_noticias_aneel
  print(len(list_noticias))

In [19]:
for index, x in enumerate(list_noticias):
    x = str(x)
    list_noticias[index] = ' '.join(word for word in x.split() if word not in lista_palavras_comuns)

In [20]:
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

dados_palavras = list(sent_to_words(list_noticias))

In [21]:
# Criando dicionário
id2word = corpora.Dictionary(dados_palavras)

# Criando corpus
texts = dados_palavras

# Frequencia do documento do termo
corpus = [id2word.doc2bow(text) for text in texts]

# visualizando
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [22]:
# Construindo LDA Model
# Este processamento pode demorar
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=2022,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

doc_lda = lda_model[corpus]

A saída de streaming foi truncada nas últimas 5000 linhas.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np

In [23]:
pprint(lda_model.print_topics())

[(0,
  '0.031*"pandemia" + 0.031*"governo" + 0.031*"aprova" + 0.026*"vai" + '
  '0.024*"transmissao" + 0.022*"economia" + 0.018*"camara" + 0.016*"amapa" + '
  '0.016*"base" + 0.014*"solar"'),
 (1,
  '0.073*"luz" + 0.040*"brasil" + 0.032*"amarela" + 0.032*"bandeira" + '
  '0.028*"barata" + 0.021*"valor" + 0.019*"ficar" + 0.015*"rio" + '
  '0.015*"tarifa" + 0.013*"crise"'),
 (2,
  '0.026*"leilao" + 0.025*"subestacao" + 0.020*"deve" + 0.016*"maior" + '
  '0.015*"gas" + 0.014*"inflacao" + 0.012*"senado" + 0.012*"cidadao" + '
  '0.012*"deixa" + 0.011*"noticias"'),
 (3,
  '0.025*"setor" + 0.014*"novo" + 0.014*"investimentos" + 0.013*"ambiental" + '
  '0.012*"eletrico" + 0.012*"procon" + 0.011*"metas" + 0.009*"zona" + '
  '0.009*"leiloes" + 0.009*"explica"'),
 (4,
  '0.034*"macapa" + 0.023*"diz" + 0.020*"apos" + 0.017*"energisa" + '
  '0.017*"pais" + 0.016*"apagao" + 0.013*"publica" + 0.012*"transformador" + '
  '0.011*"regiao" + 0.011*"covid"')]


In [24]:
# Visualização dos tópicos em clusters
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.113847  0.182128       1        1  22.776434
0      0.087608 -0.300466       2        1  21.849109
2     -0.324616  0.004332       3        1  18.993502
3      0.063825  0.051181       4        1  18.292392
1      0.059336  0.062824       5        1  18.088564, topic_info=           Term          Freq         Total Category  logprob  loglift
173         luz  11901.000000  11901.000000  Default  30.0000  30.0000
112      brasil   6546.000000   6546.000000  Default  29.0000  29.0000
73       macapa   7087.000000   7087.000000  Default  28.0000  28.0000
10878  pandemia   6152.000000   6152.000000  Default  27.0000  27.0000
134     governo   6116.000000   6116.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
13      reduzir    700.296800    701.134860   Topic5  -5.4544   1.7087
30     servicos    690.018172    690.869621   Topic5  -5.4692   1.7087
169     amarela   5253.480451   5307.663885   Topic5  -3.4393   1.6996
170    bandeira   5201.325826   6019.988374   Topic5  -3.4493   1.5637
61       aderir    759.594089    838.180587   Topic5  -5.3731   1.6114

[192 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
6306      1  0.999258  aciona
255       2  0.999253   acoes
1442      3  0.998338  acordo
61        1  0.025054  aderir
61        2  0.016703  aderir
...     ...       ...     ...
513       5  0.999745   valor
88        1  0.999538    veja
1412      4  0.999253   venda
226       2  0.999732   volta
835       4  0.999352    zona

[194 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 3, 4, 2])

In [ ]:
# salvar a visualização em arquivo HTML
pyLDAvis.save_html(vis, 'lda_5topicos_manchetes.html')

###Bigramas e Trigramas

In [25]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
list_noticias_trigramas = []
for index, x in enumerate(list_noticias):
    x = str(x)
    if (len(x.split(' ')) >= 3):
      tokens = nltk.word_tokenize(x)
      lista_trigramas = list(nltk.trigrams(tokens))
      texto_palavras_trigramas = '';
      for i, t in enumerate(lista_trigramas):
        texto_palavras_trigramas += str(t[0]) + '_' + str(t[1]) + '_' + str(t[2]) + ' '
      list_noticias_trigramas.append(texto_palavras_trigramas)

In [28]:
list_noticias_trigramas[0:10]

['atila_diminui_orçamento diminui_orçamento_arsep orçamento_arsep_aumenta arsep_aumenta_sama ',
 'juro_menor_ajuda menor_ajuda_reduzir ajuda_reduzir_custos reduzir_custos_industriais custos_industriais_editorial industriais_editorial_econômico ',
 'vem_brazil_water brazil_water_week water_week_evento week_evento_internacional evento_internacional_transformará internacional_transformará_capital transformará_capital_mundial capital_mundial_água ',
 'polo_multissetorial_serviços multissetorial_serviços_vai serviços_vai_mudar vai_mudar_cara mudar_cara_bairro cara_bairro_recife ',
 'polo_multissetorial_serviços multissetorial_serviços_vai serviços_vai_mudar vai_mudar_cara mudar_cara_bairro cara_bairro_recife ',
 'aquario_vira_referência vira_referência_limpa referência_limpa_projeto limpa_projeto_faro projeto_faro_energy ',
 'aquario_vira_referência vira_referência_limpa referência_limpa_projeto limpa_projeto_faro projeto_faro_energy ',
 'saiba_quais_projetos quais_projetos_concessões proje

In [29]:
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

dados_palavras = list(sent_to_words(list_noticias_trigramas))

In [30]:
# Criando dicionário
id2word = corpora.Dictionary(dados_palavras)

# Criando corpus
texts = dados_palavras

# Frequencia do documento do termo
corpus = [id2word.doc2bow(text) for text in texts]

# visualizando
print(corpus[:1])

[[]]


In [31]:
# Construindo LDA Model
# Este processamento pode demorar
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=2022,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

doc_lda = lda_model[corpus]

A saída de streaming foi truncada nas últimas 5000 linhas.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np

In [32]:
pprint(lda_model.print_topics())

[(0,
  '0.118*"volta_ficar_luz" + 0.055*"oleo_pre_sal" + 0.054*"pre_sal_chega" + '
  '0.043*"amapa_apos_novo" + 0.033*"luz_ficara_cara" + 0.033*"luz_carne_fazem" '
  '+ 0.023*"fgts_via_pix" + 0.017*"para_nega_plano" + 0.011*"rio_grande_sul" + '
  '0.010*"mme_lanca_plano"'),
 (1,
  '0.138*"luz_deve_ficar" + 0.050*"ruddy_valor_rio" + 0.019*"extra_cada_kwh" + '
  '0.017*"solar_gas_covid" + 0.012*"gasto_vlt_daria" + 0.010*"gas_pre_sal" + '
  '0.010*"deve_ter_alta" + 0.010*"consumo_diz_ons" + 0.010*"semana_deve_ter" + '
  '0.010*"uso_gas_pre"'),
 (2,
  '0.134*"podera_paga_pix" + 0.050*"veja_vai_mudar" + 0.050*"vai_mudar_pesar" '
  '+ 0.027*"luz_fica_barata" + 0.015*"custo_cada_kwh" + '
  '0.014*"paga_pix_brasil" + 0.010*"vai_pagar_menos" + 0.010*"pagar_menos_luz" '
  '+ 0.009*"luz_cpf_errado" + 0.009*"diz_belo_monte"'),
 (3,
  '0.071*"vao_pagar_luz" + 0.065*"pagar_luz_amapa" + 0.055*"nova_lei_gas" + '
  '0.048*"volta_ficar_luz" + 0.023*"veja_vai_mudar" + 0.023*"vai_mudar_pesar" '
  '+ 0.019

In [33]:
# Visualização dos tópicos em clusters
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.077496 -0.068010       1        1  25.486562
3      0.118888 -0.098208       2        1  25.309551
2     -0.133573 -0.137369       3        1  19.698417
0      0.230732  0.081560       4        1  16.601678
1     -0.138550  0.222027       5        1  12.903792, topic_info=                 Term        Freq       Total Category  logprob  loglift
2759   luz_deve_ficar  453.000000  453.000000  Default  30.0000  30.0000
4520  podera_paga_pix  763.000000  763.000000  Default  29.0000  29.0000
7957  volta_ficar_luz  818.000000  818.000000  Default  28.0000  28.0000
2118    vao_pagar_luz  459.000000  459.000000  Default  27.0000  27.0000
7790  pagar_luz_amapa  421.000000  421.000000  Default  26.0000  26.0000
...               ...         ...         ...      ...      ...      ...
7985  media_sin_subiu   15.541930   46.807165   Topic5  -5.3531   0.9452
4520  podera_paga_pix   20.099837  763.302214   Topic5  -5.0960  -1.5893
7983  pdi_call_center   15.113083   46.239906   Topic5  -5.3811   0.9294
7982  ccee_faz_ultimo   15.113090   95.334870   Topic5  -5.3811   0.2058
7625   cai_angra_reis   14.699947   44.372896   Topic5  -5.4088   0.9429

[258 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
1791      1  0.254817   ale_aprova_lei
1791      2  0.738971   ale_aprova_lei
7505      1  0.151343    alta_diz_ibge
7505      2  0.799958    alta_diz_ibge
7505      3  0.043241    alta_diz_ibge
...     ...       ...              ...
7957      5  0.017106  volta_ficar_luz
5523      3  0.747967    votar_lei_gas
5523      5  0.232128    votar_lei_gas
7974      5  0.976322   zfm_troca_oleo
7989      3  0.981183  zona_sul_teatro

[311 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 3, 1, 2])

In [ ]:
# salvar a visualização em arquivo HTML
pyLDAvis.save_html(vis, 'lda_5topicos_trigramas_final.html')